In [1]:
import numpy as np
import itertools as it

import allel
import pandas as pd

from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

from datetime import datetime

import collections
def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
from tools.ABC_utilities import read_demofile, demos_to_SLiM

batch= 'pm13_4a_1m_median'
recipe_dir= 'Recipes/demos_mat/'


demo_file= 'demos/test.txt'
template= 'Recipes/demos_mat/template_simple.slim'

anc_r= '0'
Nsamp= 1
sizes= 1000
burnin= 5e4
med_samp= True
rescale_dict= {}

directed= False
M_convert= True


tree, demo_data= read_demofile(demo_file)

In [3]:
tree

{'(1,2,3)': {'1': '1', '2': '2', 'node': '3'},
 '(3,4,2)': {'3': '3', '4': '4', 'node': '2'},
 'node': '1'}

In [4]:
demo_data


defaultdict(<function tools.ABC_utilities.recursively_default_dict()>,
            {'N': defaultdict(<function tools.ABC_utilities.recursively_default_dict()>,
                         {'1': array([800, 800, 800]),
                          '2': array([600, 600, 600]),
                          '3': array([400, 400, 400]),
                          '4': array([200, 200, 200]),
                          'T1': array([800, 800, 800]),
                          'T2': array([500, 500, 500]),
                          '0': array([1000, 1000, 1000])}),
             'T': defaultdict(<function tools.ABC_utilities.recursively_default_dict()>,
                         {'1': array([400, 400, 400]),
                          '2': array([1500, 1500, 1500]),
                          '3': array([750, 750, 750])}),
             'M': defaultdict(<function tools.ABC_utilities.recursively_default_dict()>,
                         {})})

In [5]:
from tools.ABC_utilities import tree_fill_list,get_tree_nodes

tree_summ= get_tree_nodes({anc_r:tree},nodes=[],edges= [],leaves= [])
recipe_dir= '/'.join(template.split('/')[:-1]) + '/'
#
anc_name= anc_r
anc_size= demo_data['N'][anc_r]

times_order= [0] + sorted(demo_data['T'].keys(),reverse= True)
int_sizes= [x for x in demo_data['N'].keys() if x[0] == 'T']
int_sizes= sorted(int_sizes)

tree_demo= tree_fill_list(times_order,tree, demo_data, tree_summ, int_sizes= int_sizes,demo_tape= [])
###

pops= tree_summ['leaves']

sizes= [sizes] * len(pops)

tree_demo= [{z:g for z,g in x.items() if len(g)} for x in tree_demo]

###

In [6]:
tree_summ

{'nodes': ['0', '(1,2,3)', '1', '2', '(3,4,2)', '3', '4'],
 'edges': [('0', '(1,2)'),
  ('0', '(3,4)'),
  ('(1,2)', '1'),
  ('(1,2)', '2'),
  ('(3,4)', '3'),
  ('(3,4)', '4')],
 'leaves': ['1', '2', '3', '4']}

In [7]:
tree_demo

[{'node': '3',
  'T': array([750, 750, 750]),
  'N': {'(3,4)': array([500, 500, 500]), '(1,2)': array([800, 800, 800])}},
 {'node': '1',
  'T': array([400, 400, 400]),
  'N': {'1': array([800, 800, 800]), '2': array([600, 600, 600])}},
 {'node': '2',
  'T': array([1500, 1500, 1500]),
  'N': {'3': array([400, 400, 400]), '4': array([200, 200, 200])}}]

In [8]:
from tools.ABC_utilities import (
    sample_dist_beta, return_replica
)
sample_func= sample_dist_beta

anc_sample= sample_func(Nsamp,*anc_size,func= int,func_args= [],med_samp= med_samp)
anc_sample

D:\GitHub\MCM\Pop_gen_PA\tools\ABC_utilities.py:357: RuntimeWarning:

invalid value encountered in long_scalars



[1000]

### Between populations. 

In [12]:
from tools.sfs_utilities_II import (
    single_gen_matrix_v2, freq_progr_func
)

from tools.ne_util import (
    theta_constant, theta_exp
)

In [13]:
scale_sim= False
burnin= 500

seqL= 1e6

muG= 1.08e-8
idx= 0
s= 0 # selection coefficient.
ploidy= 2 # ploidy.


In [14]:

### ANCESTRAL POP
sample_func= sample_dist_beta

theta_dict= {
    'func': theta_exp,#theta_constant,
    'kargs': {'rate': 0.003}
}

asize= anc_sample[idx]

### future function
Ne= asize
T= burnin
muNuc= muG

if scale_sim:
    scale_gen= model_dict[pop_select]['scale']
    muNuc= muNuc / scale_gen
    Ne= int(Ne * scale_gen)

##### get number of mutations
mu= muNuc * seqL
Theta= 4 * Ne * mu
rate_Pmut= Theta / 2

MRCA= T / Ne / 2

print("branch length: {}".format(MRCA))

Pexp= 2 * Ne * MRCA * Theta / 2 ## Multiply the standard poisson rate by 2Ne to do all the pop.

Poisson_conv= theta_dict['func'](T,Ne= Pexp,**theta_dict['kargs'])
muts= np.random.poisson(Poisson_conv,1)[0]
print('muts: {}'.format(muts))
#####

freq_ar, fixed_tally, array_spec= freq_progr_func(theta_dict,Ne= Ne,
                                      T= T,ploidy= ploidy, s= s, return_spec= True)



branch length: 0.25
muts: 48339


MemoryError: 

In [ ]:
fsul, fixed= get_fixedtally_v2(fixed_tally)


In [201]:
len(array_spec)

501

In [ ]:
replic= [return_replica(x,sample_func=sample_func,rescale_dict= rescale_dict,med_samp= med_samp) for x in tree_demo]
replic= {x['node']: x for x in replic}
replic